# In case, the file import data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install libraries
!pip install mne
!pip install pyriemann

     |████████████████████████████████| 7.4 MB 4.1 MB/s 
     |████████████████████████████████| 42 kB 447 kB/s 
  Created wheel for pyriemann: filename=pyriemann-0.2.7-py2.py3-none-any.whl size=49770 sha256=c33e5c7e568853410bd8b538a9f1b03b95f85e0520e41a65c9c2c07c900eb6bf
  Stored in directory: /root/.cache/pip/wheels/5c/b7/55/27dcb08ed8fb58da8c1be108c23928ffb9125c9c1da2ddfb53
Successfully built pyriemann


# Reference paper
- A multi-subject, multi-modal human neuroimaging dataset
    - https://github.com/bids-standard/bids-examples/tree/master/ds000117
    - https://openneuro.org/datasets/ds000117/versions/1.0.5
    - https://www.nature.com/articles/sdata20151
- Notes
    - Rename and type of channels
        - EEG061 = HEOG
        - EEG062 = VEOG
        - EEG063 = ECG
    - In README
        - There is an explanation of triggers

# Import libraries and read files

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import mne
from mne import find_events, Epochs, pick_types, read_evokeds
from mne.preprocessing import ICA

import pywt
import scipy
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)
from sklearn.neighbors import KDTree

import seaborn as sns

import os
import re

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

## Hyperparameters

In [ ]:
"""
Calling a function of artifact removal technique according to 
EOG_ref and ECG_ref parameters

EOG_ref has 4 options
1) None
2) 'wICA_without_ref'
3) 'wICA_with_ref'
4) 'EEGANet'

ECG_ref has 3 options
1) None
2) ICA_with_ref
3) wICA_with_ref
"""

sfreq_after_epoch = 200
EOG_ref = 'wICA_with_ref'
ECG_ref = 'ICA_with_ref'

In [ ]:
ch_name_dic ={'EEG001': 'PO9', 'EEG002': 'Fpz', 'EEG003': 'PO10', 'EEG004': 'AF7', 
              'EEG005': 'AF3', 'EEG006': 'AFz', 'EEG007': 'AF4', 'EEG008': 'AF8', 'EEG009': 'F7', 
              'EEG010': 'F5', 'EEG011': 'F3', 'EEG012': 'F1', 'EEG013': 'Fz', 'EEG014': 'F2', 
              'EEG015': 'F4', 'EEG016': 'F6', 'EEG017': 'F8', 'EEG018': 'FT9', 'EEG019': 'FT7', 
              'EEG020': 'FC5', 'EEG021': 'FC3', 'EEG022': 'FC1', 'EEG023': 'FCz', 'EEG024': 'FC2', 
              'EEG025': 'FC4', 'EEG026': 'FC6', 'EEG027': 'FT8', 'EEG028': 'FT10', 'EEG029': 'T9', 
              'EEG030': 'T7', 'EEG031': 'C5', 'EEG032': 'C3', 'EEG033': 'C1', 'EEG034': 'Cz', 
              'EEG035': 'C2', 'EEG036': 'C4', 'EEG037': 'C6', 'EEG038': 'T8', 'EEG039': 'T10', 
              'EEG040': 'TP9', 'EEG041': 'TP7', 'EEG042': 'CP5', 'EEG043': 'CP3', 'EEG044': 'CP1', 
              'EEG045': 'CPz', 'EEG046': 'CP2', 'EEG047': 'CP4', 'EEG048': 'CP6', 'EEG049': 'TP8', 
              'EEG050': 'TP10', 'EEG051': 'P9', 'EEG052': 'P7', 'EEG053': 'P5', 'EEG054': 'P3', 
              'EEG055': 'P1', 'EEG056': 'Pz', 'EEG057': 'P2', 'EEG058': 'P4', 'EEG059': 'P6', 
              'EEG060': 'P8', 'EEG065': 'P10', 'EEG066': 'PO7', 'EEG067': 'PO3', 'EEG068': 'POz',  
              'EEG069': 'PO4', 'EEG070': 'PO8', 'EEG071': 'O1', 'EEG072': 'Oz', 'EEG073': 'O2', 'EEG074': 'Iz'}

event_id = {'Famous':0,'Unfamiliar':1,'Scrambled':2} 

In [ ]:
def compute_kurtosis(data):
    
    """Kurtosis of the data (per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **kurtosis**
    """
    
    ndim = data.ndim
    return scipy.stats.kurtosis(data, axis=ndim - 1, fisher=False)
def compute_kurtosis(data):
    
    """Kurtosis of the data (per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **kurtosis**
    """
    
    ndim = data.ndim
    return scipy.stats.kurtosis(data, axis=ndim - 1, fisher=False)

def _app_samp_entropy_helper(data, emb, metric='chebyshev',
                             approximate=True):
    """Utility function for `compute_app_entropy`` and `compute_samp_entropy`.
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with KDTree. The list of available
        metric functions is given by: ``KDTree.valid_metrics``.
    approximate : bool (default: True)
        If True, the returned values will be used to compute the
        Approximate Entropy (AppEn). Otherwise, the values are used to compute
        the Sample Entropy (SampEn).
    Returns
    -------
    output : ndarray, shape (n_channels, 2)
    """
    _all_metrics = KDTree.valid_metrics
    if metric not in _all_metrics:
        raise ValueError('The given metric (%s) is not valid. The valid '
                         'metric names are: %s' % (metric, _all_metrics))
    n_channels, n_times = data.shape
    phi = np.empty((n_channels, 2))
    for j in range(n_channels):
        r = 0.2 * np.std(data[j, :], axis=-1, ddof=1)
        # compute phi(emb, r)
        _emb_data1 = _embed(data[j, None], emb, 1)[0, :, :]
        if approximate:
            emb_data1 = _emb_data1
        else:
            emb_data1 = _emb_data1[:-1, :]
        count1 = KDTree(emb_data1, metric=metric).query_radius(
            emb_data1, r, count_only=True).astype(np.float64)
        # compute phi(emb + 1, r)
        emb_data2 = _embed(data[j, None], emb + 1, 1)[0, :, :]
        count2 = KDTree(emb_data2, metric=metric).query_radius(
            emb_data2, r, count_only=True).astype(np.float64)
        if approximate:
            phi[j, 0] = np.mean(np.log(count1 / emb_data1.shape[0]))
            phi[j, 1] = np.mean(np.log(count2 / emb_data2.shape[0]))
        else:
            phi[j, 0] = np.mean((count1 - 1) / (emb_data1.shape[0] - 1))
            phi[j, 1] = np.mean((count2 - 1) / (emb_data2.shape[0] - 1))
    return phi


def compute_app_entropy(data, emb=2, metric='chebyshev'):
    """Approximate Entropy (AppEn, per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with
        :class:`~sklearn.neighbors.KDTree`. The list of available
        metric functions is given by: ``KDTree.valid_metrics``.
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **app_entropy**. See [1]_.
    References
    ----------
    .. [1] Richman, J. S. et al. (2000). Physiological time-series analysis
           using approximate entropy and sample entropy. American Journal of
           Physiology-Heart and Circulatory Physiology, 278(6), H2039-H2049.
    """
    phi = _app_samp_entropy_helper(data, emb=emb, metric=metric,
                                   approximate=True)
    return np.subtract(phi[:, 0], phi[:, 1])


def compute_samp_entropy(data, emb=2, metric='chebyshev'):
    
    """Sample Entropy (SampEn, per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with KDTree. The list of available
        metric functions is given by: `KDTree.valid_metrics`.
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **samp_entropy**. See [1]_.
    References
    ----------
    .. [1] Richman, J. S. et al. (2000). Physiological time-series analysis
           using approximate entropy and sample entropy. American Journal of
           Physiology-Heart and Circulatory Physiology, 278(6), H2039-H2049.
    """
    phi = _app_samp_entropy_helper(data, emb=emb, metric=metric,
                                   approximate=False)
    if np.allclose(phi[:, 0], 0) or np.allclose(phi[:, 1], 0):
        raise ValueError('Sample Entropy is not defined.')
    else:
        return -np.log(np.divide(phi[:, 1], phi[:, 0]))
    
def _embed(x, d, tau):
    """Time-delay embedding.
    Parameters
    ----------
    x : ndarray, shape (n_channels, n_times)
    d : int
        Embedding dimension.
        The embedding dimension ``d`` should be greater than 2.
    tau : int
        Delay.
        The delay parameter ``tau`` should be less or equal than
        ``floor((n_times - 1) / (d - 1))``.
    Returns
    -------
    output : ndarray, shape (n_channels, n_times - (d - 1) * tau, d)
    """
    tau_max = np.floor((x.shape[1] - 1) / (d - 1))
    if tau > tau_max:
        warn('The given value (%s) for the parameter `tau` exceeds '
             '`tau_max = floor((n_times - 1) / (d - 1))`. Using `tau_max` '
             'instead.' % tau)
        _tau = tau_max
    else:
        _tau = int(tau)
    x = x.copy()
    X = np.lib.stride_tricks.as_strided(
        x, (x.shape[0], x.shape[1] - d * _tau + _tau, d),
        (x.strides[-2], x.strides[-1], x.strides[-1] * _tau))
    return X
def _app_samp_entropy_helper(data, emb, metric='chebyshev',
                             approximate=True):
    """Utility function for `compute_app_entropy`` and `compute_samp_entropy`.
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with KDTree. The list of available
        metric functions is given by: ``KDTree.valid_metrics``.
    approximate : bool (default: True)
        If True, the returned values will be used to compute the
        Approximate Entropy (AppEn). Otherwise, the values are used to compute
        the Sample Entropy (SampEn).
    Returns
    -------
    output : ndarray, shape (n_channels, 2)
    """
    _all_metrics = KDTree.valid_metrics
    if metric not in _all_metrics:
        raise ValueError('The given metric (%s) is not valid. The valid '
                         'metric names are: %s' % (metric, _all_metrics))
    n_channels, n_times = data.shape
    phi = np.empty((n_channels, 2))
    for j in range(n_channels):
        r = 0.2 * np.std(data[j, :], axis=-1, ddof=1)
        # compute phi(emb, r)
        _emb_data1 = _embed(data[j, None], emb, 1)[0, :, :]
        if approximate:
            emb_data1 = _emb_data1
        else:
            emb_data1 = _emb_data1[:-1, :]
        count1 = KDTree(emb_data1, metric=metric).query_radius(
            emb_data1, r, count_only=True).astype(np.float64)
        # compute phi(emb + 1, r)
        emb_data2 = _embed(data[j, None], emb + 1, 1)[0, :, :]
        count2 = KDTree(emb_data2, metric=metric).query_radius(
            emb_data2, r, count_only=True).astype(np.float64)
        if approximate:
            phi[j, 0] = np.mean(np.log(count1 / emb_data1.shape[0]))
            phi[j, 1] = np.mean(np.log(count2 / emb_data2.shape[0]))
        else:
            phi[j, 0] = np.mean((count1 - 1) / (emb_data1.shape[0] - 1))
            phi[j, 1] = np.mean((count2 - 1) / (emb_data2.shape[0] - 1))
    return phi


def compute_app_entropy(data, emb=2, metric='chebyshev'):
    """Approximate Entropy (AppEn, per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with
        :class:`~sklearn.neighbors.KDTree`. The list of available
        metric functions is given by: ``KDTree.valid_metrics``.
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **app_entropy**. See [1]_.
    References
    ----------
    .. [1] Richman, J. S. et al. (2000). Physiological time-series analysis
           using approximate entropy and sample entropy. American Journal of
           Physiology-Heart and Circulatory Physiology, 278(6), H2039-H2049.
    """
    phi = _app_samp_entropy_helper(data, emb=emb, metric=metric,
                                   approximate=True)
    return np.subtract(phi[:, 0], phi[:, 1])


def compute_samp_entropy(data, emb=2, metric='chebyshev'):
    
    """Sample Entropy (SampEn, per channel).
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    emb : int (default: 2)
        Embedding dimension.
    metric : str (default: chebyshev)
        Name of the metric function used with KDTree. The list of available
        metric functions is given by: `KDTree.valid_metrics`.
    Returns
    -------
    output : ndarray, shape (n_channels,)
    Notes
    -----
    Alias of the feature function: **samp_entropy**. See [1]_.
    References
    ----------
    .. [1] Richman, J. S. et al. (2000). Physiological time-series analysis
           using approximate entropy and sample entropy. American Journal of
           Physiology-Heart and Circulatory Physiology, 278(6), H2039-H2049.
    """
    phi = _app_samp_entropy_helper(data, emb=emb, metric=metric,
                                   approximate=False)
    if np.allclose(phi[:, 0], 0) or np.allclose(phi[:, 1], 0):
        raise ValueError('Sample Entropy is not defined.')
    else:
        return -np.log(np.divide(phi[:, 1], phi[:, 0]))
    
def _embed(x, d, tau):
    """Time-delay embedding.
    Parameters
    ----------
    x : ndarray, shape (n_channels, n_times)
    d : int
        Embedding dimension.
        The embedding dimension ``d`` should be greater than 2.
    tau : int
        Delay.
        The delay parameter ``tau`` should be less or equal than
        ``floor((n_times - 1) / (d - 1))``.
    Returns
    -------
    output : ndarray, shape (n_channels, n_times - (d - 1) * tau, d)
    """
    tau_max = np.floor((x.shape[1] - 1) / (d - 1))
    if tau > tau_max:
        warn('The given value (%s) for the parameter `tau` exceeds '
             '`tau_max = floor((n_times - 1) / (d - 1))`. Using `tau_max` '
             'instead.' % tau)
        _tau = tau_max
    else:
        _tau = int(tau)
    x = x.copy()
    X = np.lib.stride_tricks.as_strided(
        x, (x.shape[0], x.shape[1] - d * _tau + _tau, d),
        (x.strides[-2], x.strides[-1], x.strides[-1] * _tau))
    return X

In [ ]:
# Method for finding threshold, 'Thresholding selection rule'
def sqtwolog_threshold(wavelet_coeffs):
    """
    Universal thresholding
    """
    denom = scipy.stats.norm.ppf(0.75) # the constant value for Gaussian noise (0.6745)
    var = np.median(np.abs(wavelet_coeffs)) / denom
    N = len(wavelet_coeffs)
    thre = np.sqrt(var) * np.sqrt(2 * np.log(N))
    return thre

def rigsure_threshold(wavelet_coeffs):
    var = np.std(wavelet_coeffs)
    N = len(wavelet_coeffs)
    sqr_coeffs = []
    for coeff in wavelet_coeffs:
        sqr_coeffs.append(np.power(coeff, 2))
    sqr_coeffs.sort()
    pos = 0
    r = 0
    for idx, sqr_coeff in enumerate(sqr_coeffs):
        new_r = (N - 2 * (idx + 1) + (N - (idx + 1))*sqr_coeff + sum(sqr_coeffs[0:idx+1])) / N
        if r == 0 or r > new_r:
            r = new_r
            pos = idx
    thre = np.sqrt(var) * np.sqrt(sqr_coeffs[pos])
    return thre

def heursure_threshold(wavelet_coeffs):
    N = len(wavelet_coeffs)
    s = 0
    for coeff in wavelet_coeffs:
        s += np.power(coeff, 2)
    theta = (s - N) / N
    # It is divide by ...
    miu = np.power(np.log2(N), 3/2) / np.power(N, 1/2)
    if theta < miu:
        return sqtwolog_threshold(wavelet_coeffs)
    else:
        return min(sqtwolog_threshold(wavelet_coeffs), rigsure_threshold(wavelet_coeffs))
    
def statistical_threshold(wavelet_coeffs):
    """
    Put wavelet coefficients of one level to the function
    Return the statistical threshold value of that level
    """
    threshold_value = 1.5 * np.std(wavelet_coeffs)
    return threshold_value

In [ ]:
def denoised_with_DiscreteWavelet(oneCH_EEG, 
                                  motherWavelet="bior3.9", decLevel=8, 
                                  thresholdSelectionRule="statistical_threshold", thresholdingFunction="hard",
                                  lcoeffLevel=1, rcoeffLevel=5):
    """
    contEEG is a dataset containing contaminated EEG (54 subjects each has 19 EEG channels)
    motherWavelet = 'bior3.9' (Paper)
    decLevel = 8 (Calculate)
    thresholdSelectionRule = "heursure_threshold" or "statistical_threshold"
    thresholdingFunction = "soft" or "hard" thresholding
    lcoeffLevel = the coefficients corresponding to lower frequency bands that we want to threshold/denoise
    rcoeffLevel = the coefficients corresponding to upper frequency bands that we want to threshold/denoise
    """
    
    denoised_coeffs = [] # Prepare for contain denoise coefficients
    coeffs = pywt.wavedec(oneCH_EEG, wavelet=motherWavelet, level=decLevel)
    for ix, coeff in enumerate(coeffs):
        # thresholding has been done over the cD from level 8 up to level 3 (Cover OAs freq. band/related coefficients)
        threshold_value = None
        if ix in range(lcoeffLevel,rcoeffLevel):
            if thresholdSelectionRule == "statistical_threshold":
                threshold_value = statistical_threshold(wavelet_coeffs=coeff)
            else:
                threshold_value = heursure_threshold(wavelet_coeffs=coeff)

            # According to the paper
            # wavelet coefficient (wc) is removed if np.abs(wc) > threshold value
            denoised_coeff = np.where(np.abs(coeff) > threshold_value, 0, coeff)
            denoised_coeffs.append(denoised_coeff)

        else:
            denoised_coeffs.append(coeff)  
        
    denoised_oneCH_EEG = pywt.waverec(denoised_coeffs, wavelet=motherWavelet)
    return denoised_oneCH_EEG

In [ ]:
def signals_to_sources(ica, data):
    """Compute sources from data (operates inplace)."""
    _data = data.copy()
    _data = ica._pre_whiten(_data)
    if ica.pca_mean_ is not None:
        _data -= ica.pca_mean_[:, None]

    # Apply unmixing
    pca_data = np.dot(ica.unmixing_matrix_,
                    ica.pca_components_[:ica.n_components_,:])
    # Apply PCA
    sources = np.dot(pca_data, _data)
    return sources

def sources_to_signals(ica, sources):
    """Compute Mixed signals from sources (operates inplace)."""
    _sources = sources.copy()
    # Apply mixing (Inverse of product)
    inv_pca_data = np.dot(np.linalg.inv(ica.pca_components_)[:,:ica.n_components_],ica.mixing_matrix_)
    # Apply invsersed PCA
    _data = np.dot(inv_pca_data, _sources)

    if ica.pca_mean_ is not None:
        _data += ica.pca_mean_[:, None]
    _data *= ica.pre_whitener_
    return _data

In [ ]:
def ICA_on_raw_N170(raw, EOG_ref=False, ECG_ref=False, 
                    n_component=None):

    # Number of each type of channel
    n_eeg = raw.get_channel_types().count('eeg')
    n_eog = raw.get_channel_types().count('eog')
    n_ecg = raw.get_channel_types().count('ecg')

    # Remove artifact
    ica = ICA(n_components=n_component, max_iter='auto', 
            method='infomax',fit_params=dict(extended=True),
            random_state=32, verbose=False)
    ica.fit(raw)
    ica.exclude = []
    eog_indices = []
    ecg_indices = []
    # Find which ICs match the EOG pattern
    if EOG_ref == True:
        eog_indices, eog_scores = ica.find_bads_eog(raw, verbose=False)
    # Find which ICs match the ECG pattern
    if ECG_ref == True:
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation',
                                                    threshold='auto')
    # Combined the result from finding EOG and ECG ICs
    artifact_indices = list(set(eog_indices + ecg_indices))
    ica.exclude = artifact_indices
    ica.apply(raw)

    return raw

In [ ]:
def wICA_on_raw_N170(raw, EOG_ref=False, ECG_ref=False):  
    """
    If EOG_ref == True:
        Detecting EOG artifact IC(s) depend on EOG reference channels
    If EOG_ref == True:
        Detecting EOG artifact IC(s) depend on EOG reference channels
    Else:
        Detecting artifact IC(s) by mMSE and Kurtosis
    """
    filt_raw = raw.copy()

    n_eeg = filt_raw.get_channel_types().count('eeg')
    n_eog = filt_raw.get_channel_types().count('eog')
    n_ecg = filt_raw.get_channel_types().count('ecg')

    ica = ICA(n_components=n_eeg, method="infomax", 
            fit_params=dict(extended=True), random_state=32)
    ica.fit(filt_raw)
  
    # Get ICs
    cont_ICs = ica.get_sources(filt_raw).get_data()
    cont_IC_samples = cont_ICs.shape[-1]
    # Use EOG reference channels to find which IC is EOG pattern
    if EOG_ref == True:
        eog_indices, eog_scores = ica.find_bads_eog(filt_raw, verbose=False)
        artifact_ICs_indexs = np.zeros((n_eeg), dtype=bool)
        artifact_ICs_indexs[eog_indices] = True

    else:
        # Calculate mMSE
        coarse_graining_ICs = []
        for e_IC in cont_ICs:
            
            coarse_graining_e_IC = []
            length_data = e_IC.shape[-1]
            mMSE_scale_factor = 20
            range_coarse_graining = np.arange(1, int(length_data/mMSE_scale_factor)+1, 1)

            for j in range_coarse_graining:
                # Python is zero-index
                start_point = ((j-1)*mMSE_scale_factor)
                end_point = j * mMSE_scale_factor
                coase_graining = (1/mMSE_scale_factor) * np.sum(e_IC[start_point:end_point])
                coarse_graining_e_IC.append(coase_graining)
                
            coarse_graining_ICs.append(np.array(coarse_graining_e_IC))

        mMSE_ICs = compute_samp_entropy(data=np.array(coarse_graining_ICs))

        # Calculate Kurtosis
        kurtosis_ICs= []

        for e_IC in cont_ICs:
            kurtosis_e_IC = compute_kurtosis(data=e_IC)
            kurtosis_ICs.append(kurtosis_e_IC)

        kurtosis_ICs = np.array(kurtosis_ICs)

        # Check which ICs are artifactual ICs
        # alpha level of two tailed t-test = 0.05
        alpha = 0.05
        critical_value = scipy.stats.t.ppf(q=1-alpha/2,df=int(n_eeg)-1)

        # Calculate Upper limit and Lower limit
        lower_limit= np.mean(mMSE_ICs) - ((np.std(mMSE_ICs) / np.sqrt(len(mMSE_ICs)) * critical_value))
        upper_limit= np.mean(kurtosis_ICs) - ((np.std(kurtosis_ICs) / np.sqrt(len(kurtosis_ICs))) * critical_value)

        # print(f"Check which IC is artifact IC\n {(mMSE_ICs < lower_limit) & (kurtosis_ICs > upper_limit)}")
        artifact_ICs_indexs = (mMSE_ICs < lower_limit) & (kurtosis_ICs > upper_limit)
        non_artifact_ICs_indexs = np.invert(artifact_ICs_indexs)

        artifact_ICs = cont_ICs[artifact_ICs_indexs, :]
        non_artifact_ICs = cont_ICs[non_artifact_ICs_indexs, :]


    # Denoise artifactual ICs with Wavelet
    denoised_ICs = []
    for idx_IC, IC_bool in zip(range(n_eeg), artifact_ICs_indexs):
        
        selected_IC = cont_ICs[idx_IC, :]
        
        # True if IC is artifactual IC
        if IC_bool == True:
            
            denoised_IC = denoised_with_DiscreteWavelet(selected_IC, 
                                            motherWavelet="bior3.9", decLevel=8, 
                                            thresholdSelectionRule="statistical_threshold", 
                                            thresholdingFunction="hard",
                                            lcoeffLevel=1, rcoeffLevel=5)
            denoised_ICs.append(denoised_IC[:cont_IC_samples])
        # False if IC is non-artifactual IC
        elif IC_bool == False:
            denoised_ICs.append(selected_IC[:cont_IC_samples])

    # List to Numpy
    denoised_ICs = np.array(denoised_ICs)

    # Use ECG reference channels to find which IC is ECG pattern
    if ECG_ref == True: 
        ecg_indices, ecg_scores = ica.find_bads_ecg(filt_raw, method='correlation',
                                                    threshold='auto', verbose=False)
        # For checking IC(s) is not EOG IC
        eog_indices, eog_scores = ica.find_bads_eog(filt_raw, verbose=False)

        for ecg_index in ecg_indices:
            if ecg_index not in eog_indices:
                # Zero out
                denoised_ICs[ecg_index,:] = 0

    # Report artifactual ICs
    if EOG_ref == True and ECG_ref == True:
        # wICA with EOG reference and ICA with ECG reference
        eog_indices = eog_indices
        ecg_indices = ecg_indices
        print(f'artifact ICs are {np.union1d(eog_indices, ecg_indices)}')

    elif EOG_ref == True and ECG_ref == False:
        # wICA with EOG referece
        eog_indices = eog_indices
        ecg_indices = np.array([])
        print(f'artifact ICs are {np.union1d(eog_indices, ecg_indices)}')

    elif EOG_ref == False and ECG_ref == True:
        # wICA without EOG reference and ICA with ECG reference
        eog_indices = np.argwhere(artifact_ICs_indexs == True)
        ecg_indices = ecg_indices
        print(f'artifact ICs are {np.union1d(eog_indices, ecg_indices)}')
        
    else: # EOG_ref == False and ECG_ref == False:
        eog_indices = np.argwhere(artifact_ICs_indexs == True)
        ecg_indices = np.array([])
        print(f'artifact ICs are {np.union1d(eog_indices, ecg_indices)}')

    # Mix ICs to obtain mixed signals
    denoised_EEG = sources_to_signals(ica=ica, sources=denoised_ICs)
    # HEOG,VEOG,ECG
    non_eeg = raw.copy().pick_types(eeg=False, eog=True, ecg=True, stim=True).get_data()
    # Concatenate denoised EEG and non-eeg
    denoised_EEG = np.concatenate((denoised_EEG, non_eeg))
    # Create raw object for denoised EEG
    denoised_raw = raw.copy()
    denoised_raw._data = denoised_EEG

    return denoised_raw

In [ ]:
def artifact_removal(raw, EOG_ref, ECG_ref):
    """
    Calling a function of artifact removal technique according to 
    EOG_ref and ECG_ref parameters

    EOG_ref has 4 options
    1) None
    2) 'wICA_without_ref'
    3) 'wICA_with_ref'
    4) 'EEGANet'

    ECG_ref has 3 options
    1) None
    2) ICA_with_ref
    3) wICA_with_ref
    """
    input_raw = raw.copy()
    sfreq_after_epoch = 200 # Resample or not

    if EOG_ref == None and ECG_ref == None:
        output_raw = input_raw

    elif EOG_ref == 'wICA_without_ref' and ECG_ref == None:
        output_raw = wICA_on_raw_N170(raw, EOG_ref=False, ECG_ref=False)

    elif EOG_ref == 'wICA_with_ref' and ECG_ref == None:
        output_raw = wICA_on_raw_N170(raw, EOG_ref=True, ECG_ref=False)

    elif EOG_ref == None and ECG_ref == 'ICA_with_ref':
        output_raw = ICA_on_raw_N170(raw, ECG_ref=True)

    elif EOG_ref == 'wICA_without_ref' and ECG_ref == 'ICA_with_ref':
        output_raw = wICA_on_raw_N170(raw, EOG_ref=False, ECG_ref=False)

    elif EOG_ref == 'wICA_with_ref' and ECG_ref == 'ICA_with_ref':
        output_raw = wICA_on_raw_N170(raw, EOG_ref=True, ECG_ref=False)

    elif EOG_ref == 'wICA_without_ref' and ECG_ref == 'wICA_with_ref':
        output_raw = wICA_on_raw_N170(raw, EOG_ref=False, ECG_ref=False)

    elif EOG_ref == 'wICA_with_ref' and ECG_ref == 'wICA_with_ref':
        output_raw = wICA_on_raw_N170(raw, EOG_ref=True, ECG_ref=False)

    return output_raw

## Check the folder's name using for containing pre-processed files

In [ ]:
if EOG_ref == None and ECG_ref == None:
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_without_ref' and ECG_ref == None:
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_with_ref' and ECG_ref == None:
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'EEGANet' and ECG_ref == None:
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == None and ECG_ref == 'ICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == None and ECG_ref == 'wICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_without_ref' and ECG_ref == 'ICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_with_ref' and ECG_ref == 'ICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'EEGANet' and ECG_ref == 'ICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_without_ref' and ECG_ref == 'wICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'wICA_with_ref' and ECG_ref == 'wICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

elif EOG_ref == 'EEGANet' and ECG_ref == 'wICA_with_ref':
    ARTIFACT_SUB_FOLDER = f"raw_{sfreq_after_epoch}Hz-39EEGchs-{EOG_ref}-{ECG_ref}"

print(f"Folder name is {ARTIFACT_SUB_FOLDER}")

Folder name is raw_200Hz-39EEGchs-wICA_with_ref-ICA_with_ref


# Raw

### Decomposition by DWT

From the input signal's sampling frequency was **200 Hz** and OAs occur due to eye moment and eye-blinks and have freuqncy ranges of 0-7 Hz and 8-13 Hz

- level 01: 50-100 Hz
- level 02: 25-50 Hz
- level 03: 12.5-25 Hz
- level 04: 6.25-12.5 Hz
- level 05: 3.125-6.25 Hz
- level 06: 1.5625-3.125 Hz
- level 07: 0.78125-1.5625 Hz
- level 08: 0.390625-0.78125 Hz


In [ ]:
for subID in range(1,11):
    path = f'Datasets/N170/sub-{subID:02}/ses-meg/meg'
    for runID in range(1,7):
        # Path of signal and event files
        file_fif = f'{path}/sub-{subID:02}_ses-meg_task-facerecognition_run-{runID:02}_meg.fif'

        raw = mne.io.read_raw_fif(file_fif,preload=True, verbose=False)
        raw.set_channel_types({'EEG061':'eog','EEG062':'eog','EEG063':'ecg','EEG064':'misc'})
        # raw.plot(block=True)
        raw.rename_channels({'EEG061': 'HEOG',
                             'EEG062': 'VEOG',
                             'EEG063': 'ECG'})
        # Change some channels'types
        raw.set_channel_types({'HEOG': 'eog',
                               'VEOG': 'eog',
                               'ECG': 'ecg',
                               'EEG064':'misc'})
        # Rename channels
        raw.rename_channels(ch_name_dic)
        # Pick channels 
        selected_ch_names = ['Fpz', 'AF3', 'AF4', 'F7', 'F3', 'Fz', 'F4', 'F8',
                            'FT9', 'FC5', 'FC1', 'FC2', 'FC6', 'FT10',
                            'T9', 'C5', 'C1', 'C2', 'C6', 'T10',
                            'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10',
                            'P9', 'P5', 'P1', 'P2', 'P6', 'P10',
                            'PO9', 'PO3', 'PO4', 'PO10',
                            'O1', 'O2', 'Iz',
                            'HEOG', 'VEOG', 'ECG', 'STI101']
        raw.pick_channels(selected_ch_names)
        
        # Read a event file
        # 3 columns (event time in samples, stim's value before event, event id)
        all_events = mne.find_events(raw,stim_channel='STI101',shortest_event=1,
                                    verbose=False) 

        # Select only stimulus-related events
        events = []
        for e_event in all_events:
            if e_event[2] in [5,6,7,13,14,15,17,18,19]:
                events.append(e_event)
        events = np.array(events)
  
        # Re-label 9 labels to 3 labels
        # Famous:1, Unfamiliar:2, Scrambled Face:3
        events = mne.merge_events(events,[5,6,7],0)
        events = mne.merge_events(events,[13,14,15],1)
        events = mne.merge_events(events,[17,18,19],2)

        # Notch filter and Bandpass
        # Remove power line
        half_sfreq = int(raw.info['sfreq'] / 2)
        raw.notch_filter(freqs=np.arange(50,half_sfreq,50))
        raw.filter(1,30,n_jobs=2,fir_design='firwin')


        # # Apply Artifact removal technique
        raw = artifact_removal(raw, EOG_ref=EOG_ref, ECG_ref=ECG_ref)
        
        # Drop bad channels 
        # Visual checking on evoked and some channels went bad after applying artifact removal techniques
        raw.drop_channels(['O1', 'O2', 'Iz'])

        picks = mne.pick_types(raw.info,meg=False,eeg=True,stim=True,eog=True,ecg=True,misc=False)
        epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.6, picks=None,
                            baseline=(-0.2, 0), reject=None, preload=True, 
                            verbose=False)
        
        # epoch_raw.drop_channels(['EEG061','EEG062','EEG063','EEG064'])
        epochs.resample(200.00)
        # epochs.rename_channels(ch_name_dic)
        # epochs.plot(block=True)
        # epochs.info['bads'] = ['EEG061','EEG062','EEG063','EEG064']
        # epochs.interpolate_bads(reset_bads=True, mode='accurate', verbose=None)

        epochs.set_eeg_reference(ref_channels='average', projection=True)
        epochs.apply_proj()
        # # Reject signals according their amplitudes
        # eeg_reject = dict(eeg=200e-6)

        # epochs.drop_bad(reject=eeg_reject, flat=None, verbose=None)

        # Plot evoked
        evoked_non = epochs['Scrambled'].average()
        evoked_non.plot(spatial_colors=True, gfp=True, time_unit='s')
        evoked_target = epochs['Famous'].average()
        evoked_target.plot(spatial_colors=True, gfp=True, time_unit='s')

        epochs.save(fname=f'Datasets/N170-preprocessed-39EEGchs/{ARTIFACT_SUB_FOLDER}/sub-{subID:02}-run-{runID:02}-eeg-epo.fif',
                    split_size='2GB',
                    overwrite=True,
                    verbose=False)
        
        # Misc
        print(f"Saved sub-{subID:02}-run-{runID:02}-eeg-epo.fif file")
        print('-'*150)
        
    #     break
    # break

### Test start

### Test stop

# Epoch and Modeling

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from mne.preprocessing import Xdawn
from pyriemann.estimation import ERPCovariances
from collections import OrderedDict
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from pyriemann.estimation import XdawnCovariances

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from mne.decoding import Vectorizer
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline

In [ ]:
for subID in range(5,11):
    path = f'Datasets/N170-preprocessed-39EEGchs/{ARTIFACT_SUB_FOLDER}/'
    fif_list = [f'{path}sub-{subID:02}-run-{runID:02}-eeg-epo.fif'
                for runID in range(1,7)]
    print(f"path: {path}")
    list_epochs = []
    for file_fif in fif_list:
        epochs = mne.read_epochs(fname=file_fif, preload=True, verbose=False)
        # Select only EEG channels for classification
        epochs.pick_types(eeg=True)
        list_epochs.append(epochs)

    # Extract features and targets
    X = epochs.get_data()
    y = epochs.events[:, -1]

    # # Select only Famous:0 and Scrambled:2 events
    # mask_FS = np.isin(element=y, test_elements=[0,2])

    # X = X[mask_FS]
    # y = y[mask_FS]

    print(X.shape, y.shape)

    # Combined Epochs
    for idx_epochs, epochs in enumerate(list_epochs):
        # mne to Numpy
        e_X = epochs.copy().pick_types(eeg=True).get_data()
        e_y = epochs.events[:, -1]
        # print(e_X.shape)
        if idx_epochs == 0:
            X = e_X
            y = e_y
        else:
            X = np.concatenate((X, e_X), axis=0)
            y = np.concatenate((y, e_y), axis=0)

    # # Select only Famous:0 and Scrambled:2 events
    # mask_FS = np.isin(element=y, test_elements=[0,2])

    # X = X[mask_FS]
    # y = y[mask_FS]

    print(X.shape, y.shape)

    lda = LDA(shrinkage='auto', solver='eigen') #Regularized LDA
    lr = LogisticRegression(penalty='l1', solver='liblinear',
                                        multi_class='auto')

    clfs = OrderedDict()

    n_components = 3

    clfs['ERP + TS + LR']= make_pipeline(ERPCovariances(estimator='oas'), 
                                        TangentSpace(), 
                                        LogisticRegression())
    # clfs['ERP + TS + MinMax + LR']= make_pipeline(ERPCovariances(estimator='oas'), 
    #                                             TangentSpace(), 
    #                                             MinMaxScaler(),
    #                                             LogisticRegression())
    clfs['ERP + TS + SVC']= make_pipeline(ERPCovariances(estimator='oas'), 
                                        TangentSpace(), 
                                        SVC(decision_function_shape='ovr'))
    # clfs['ERP + TS + MinMax + SVC']= make_pipeline(ERPCovariances(estimator='oas'), 
    #                                             TangentSpace(), 
    #                                             MinMaxScaler(),
    #                                             SVC(decision_function_shape='ovr'))
    clfs['ERP + MDM'] = make_pipeline(ERPCovariances(estimator='oas'), 
                                    MDM())
    # clfs['Xdawn + RegLDA'] = make_pipeline(XdawnCovariances(n_components, 
    #                             estimator='oas'), Vectorizer(), LogisticRegression())
    # clfs['Xdawn + MDM'] = make_pipeline(XdawnCovariances(n_components,
    #                             estimator='oas'), MDM())

    # Cross validator
    cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

    for clf in clfs:
        # Do cross-validation
        preds = np.empty(len(y))
        for train, test in cv.split(X, y):  #Xdawn takes in epoch object
            clfs[clf].fit(X[train], y[train])
            preds[test] = clfs[clf].predict(X[test])

        # Classification report
        target_names = ['0', '1', '2']
        report = classification_report(y, preds, target_names=target_names)
        print(clf)
        print(report)
        
    print('-'*150)

path: Datasets/N170-preprocessed-39EEGchs/raw_200Hz-39EEGchs-wICA_with_ref-ICA_with_ref/
(147, 36, 160) (147,)
(883, 36, 160) (883,)
ERP + TS + LR
              precision    recall  f1-score   support

           0       0.45      0.45      0.45       294
           1       0.44      0.44      0.44       296
           2       0.76      0.77      0.77       293

    accuracy                           0.55       883
   macro avg       0.55      0.55      0.55       883
weighted avg       0.55      0.55      0.55       883

ERP + TS + SVC
              precision    recall  f1-score   support

           0       0.44      0.43      0.43       294
           1       0.43      0.41      0.42       296
           2       0.78      0.81      0.80       293

    accuracy                           0.55       883
   macro avg       0.55      0.55      0.55       883
weighted avg       0.55      0.55      0.55       883

ERP + MDM
              precision    recall  f1-score   support

           

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


ERP + TS + LR
              precision    recall  f1-score   support

           0       0.39      0.40      0.39       290
           1       0.41      0.43      0.42       296
           2       0.77      0.71      0.74       297

    accuracy                           0.52       883
   macro avg       0.52      0.51      0.52       883
weighted avg       0.52      0.52      0.52       883

ERP + TS + SVC
              precision    recall  f1-score   support

           0       0.42      0.38      0.40       290
           1       0.43      0.51      0.46       296
           2       0.79      0.70      0.74       297

    accuracy                           0.53       883
   macro avg       0.54      0.53      0.54       883
weighted avg       0.55      0.53      0.54       883

ERP + MDM
              precision    recall  f1-score   support

           0       0.36      0.57      0.44       290
           1       0.41      0.24      0.30       296
           2       0.57      0.47   